# **<center> Análisis de Sentimientos de los candidatos presidenciales </br> Ecuador 2021, en la red social Twitter </center>**
<h6>
    <center> Miguel Angel Macias, Jonnathan Campoberde <br/> 
    <i>Universidad de Cuenca <br/>  <i>Facultad de Ingenieria - Escuela de Sistemas<br/> Cuenca, Ecuador</i>
    <br/> { mangel.maciasn, jonnathan.campoberde }@ucuenca.edu.ec</center>
<h6>

# **I.&emsp;Introducción**
El presente trabajo tiene como objetivo emplear de manera práctica todos los conocimientos adquiridos en el transcurso de la asignatura “Text Mining”. Se trabajará con un dataset... . Se busca analizar el ..., detectando las ... por medio de ... . Todo esto con el propósito de ... .

## **1.1.&emsp;Objetivos del Proyecto**
>El objetivo del proyecto es el Análisis de Sentimientos de los candidatos presidenciales de Ecuador 2021, para luego contrastar los resultados obtenidos con los resultados oficiales de las elecciones Posteriormente se desea responder a las preguntas definidas en 1.2.

## **1.2.&emsp;Preguntas de investigación**
>- RQ1:     ¿Qué tan precisamente los datos de Twitter reflejan el sentimiento político en un proceso electoral?
>- RQ1:     ¿En qué grado de aceptación se encuentran los candidatos dentro de la red Social Twitter?
>- RQ3:     ¿Los resultadso oficiales son parecidos a los hallados en nuestro análisis?


# **II.&emsp;TEXT CLEANING**

## **2.1.&emsp; Reading dataset**
>Para este paso...

In [1]:
#!pip install emoji

In [2]:
import warnings
warnings.filterwarnings("ignore")

#from datetime import datetime, timezone
import pytz
import numpy as np              # Para operaciones matematicas
import pandas as pd             # Manejo de datos / dataframes
import re
import emoji
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer
%matplotlib inline

In [3]:
data_raw = '../data/raw'              # directorio que contiene toda la data sin manipular
data_processed = '../data/processed'  # directorio que contiene data procesada

In [4]:
df_lasso = pd.read_csv(data_raw+"/all_tweets_lasso.csv")
df_lasso['created_at'] = pd.to_datetime(df_lasso['created_at'], utc=pytz.UTC)
df_lasso['user.created_at'] = pd.to_datetime(df_lasso['user.created_at'], utc=pytz.UTC)
df_lasso

,created_at,favorite_count,full_text,id_str,metadata.iso_language_code,retweeted_status.created_at,retweeted_status.full_text,user.created_at,user.favourites_count,user.followers_count,user.friends_count,user.id_str,user.location,user.verified,retweet_count
0,2021-02-01 00:00:00+00:00,1,#Elecciones2021Ec | ¿Las propuestas económicas...,1356029498307784705,es,NaN,NaN,2011-12-05 13:59:33+00:00,269,12757,288,429043466,Ecuador,False,0
1,2021-01-31 23:59:47+00:00,0,RT @karitupl_: Y si van a venir con lo del Fer...,1356029442645233665,es,Sun Jan 31 22:19:16 +0000 2021,"Y si van a venir con lo del Feriado Bancario, ...",2019-01-11 01:19:38+00:00,10928,31,487,1083533862066634752,"Quito, Ecuador",False,95
2,2021-01-31 23:59:30+00:00,0,RT @relicheandres: Guillermo Lasso es una cart...,1356029372596154381,es,Sun Jan 31 18:45:05 +0000 2021,Guillermo Lasso es una carta perdida. Su propi...,2012-06-06 06:19:48+00:00,18497,464,2667,600732907,NaN,False,215
3,2021-01-31 23:59:22+00:00,0,RT @fabriciovelav: AHORA: @ECUATRANSA despidió...,1356029337548558337,es,Sun Jan 31 16:08:19 +0000 2021,"AHORA: @ECUATRANSA despidió a su gerente, tras...",2017-07-03 14:34:05+00:00,10857,128,2410,881883752145793025,"Latacunga, Ecuador",False,917
4,2021-01-31 23:59:19+00:00,0,RT @farriancito: Guillermo Lasso:\r\n'Gracias ...,1356029324969836544,es,Sun Jan 31 19:17:57 +0000 2021,Guillermo Lasso:\r\n'Gracias Dalo Bucaram'\r\n...,2011-12-07 02:30:28+00:00,24803,748,1007,430359745,Guayaquil,False,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55050,2021-02-01 00:01:01+00:00,1,Ecuador se apresta para un nuevo proceso elect...,1356029751929012225,es,NaN,NaN,2015-04-11 11:36:17+00:00,652,166,1429,3155978477,"Molina de Segura, España",False,1
55051,2021-02-01 00:00:21+00:00,0,RT @FERNANDOREVCOM1: @ElFerRamirezG @Sgangoten...,1356029584580485122,es,Sun Jan 31 21:39:04 +0000 2021,"@ElFerRamirezG @SgangotenaG No al racismo, ni ...",2021-01-21 12:14:58+00:00,252,1,111,1352228070103658496,NaN,False,2
55052,2021-02-01 00:00:15+00:00,0,RT @pochoharbEC: Negativa impresión la actitud...,1356029560047996928,es,Sun Jan 31 12:36:40 +0000 2021,Negativa impresión la actitud de Isidro Romero...,2017-08-05 17:53:15+00:00,3769,16,56,893892671676321792,NaN,False,361
55053,2021-02-01 00:00:12+00:00,0,RT @savirosero: Mi abuelita toda su vida ha ap...,1356029545829314560,es,Sun Jan 31 23:22:40 +0000 2021,Mi abuelita toda su vida ha apoyado a la Izqui...,2015-03-21 04:16:03+00:00,80232,885,820,3100934092,NaN,False,8


## **2.2.&emsp; Removing Duplicates**
>lorem ipsum

In [5]:
# Count duplicates
len(df_lasso['full_text'])-len(df_lasso['full_text'].drop_duplicates())

47145

In [6]:
df_lasso_processed = df_lasso.drop_duplicates( ['full_text'], keep='first')

In [7]:
len(df_lasso_processed['full_text'])

7910

## **2.3.&emsp; Identify tweets by trolls**
>Lorem ipsum

In [8]:
def is_troll(Dt, Dc, Fc, fc, v):
    '''
    ----------------------------------
    | Variable Description           |
    ----------------------------------
    | Dt  stands for Date Tweet      |
    | Dc  stands for Date creation   |
    | Fc  stands for Followers count |
    | fc  stands for friend count    |
    | v   stands for verified        |
    ----------------------------------
    | Weight penalization            |
    ----------------------------------
    | Timeliness metric:  WD = 0.7   |
    | Followers metric:   WF = 0.2   |
    | Following metric:   Wf = 0.1   |
    ----------------------------------
    | Acceptance Threshold           |
    ----------------------------------
    | Non Troll   > 0.8              |
    | Troll       < 0.8              |
    ----------------------------------
    '''
    # If user is verified then is a real person
    if v:
        return True
    else:

        threshold = 0.7*((Dt-Dc).days)/180 + 0.2*(Fc/250) + 0.1*(fc/300)

        return threshold < 0.8


In [9]:
df_lasso_processed['is_troll'] = df_lasso_processed.apply(lambda x: is_troll(x['created_at'],
                                                         x['user.created_at'],
                                                         x['user.followers_count'],
                                                         x['user.friends_count'],
                                                         x['user.verified']), axis=1)

## **2.4.&emsp; Removing characters and Specific Stopwords**
>lorem ipsum

In [10]:
def clean_text(text):
    """Function to clean text extracted from the Internet with html tags
    
    Parameters
    ----------
    text: str 
      original string
    
    Return
    ------
    text: str
      modified initial string
    """

    text = text.lower() # lowercase text
    text = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", text) #Remove http links
    text = EMOJI.sub(r'', text)
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

In [11]:
# filtrado de caracteres y stopwords
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@#:,;¿?!¡-]')
EMOJI = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)

STOPWORDS = ('rt')  # Adding new stopwords to the set, 'cause theese words repeat in all docs

df_lasso_processed['full_text'] = df_lasso_processed["full_text"].apply(clean_text)

### _Luego de remover los caracteres especiales pueden existir Tweets vacios, debido a que estos solo contenian emotes o enlaces en su interior. Es por esta razon que se buscan estos espacios en blanco y se los quita_

In [12]:
df_lasso_processed[df_lasso_processed['full_text']==""]["full_text"].count()

26

In [13]:
df_lasso_processed.drop(df_lasso_processed.loc[df_lasso_processed["full_text"]==""].index, inplace=True)

In [14]:
df_lasso_processed[df_lasso_processed['full_text']==""]["full_text"].count()

0

In [15]:
df_lasso_processed.to_csv(data_processed+'/lasso_clean.csv', 
                        index=False, header=True, chunksize=10000)